# Redcapy Unit Testing
##### WARNING: This notebook contains unit tests. While they may be used liberally in a development stage project, import tests should NOT be used in a production project.
- Variable names should reflect names within your Redcap project in order for the tests to succeed. This is merely a framework to be modified for a given project for which you have API access.

### Imports

In [1]:
import io, os, sys, types
import unittest
import json
import pprint
import hashlib
import time

from redcap.redcapy import Redcapy
from datetime import datetime, date, timedelta
from unittest import TestLoader, TextTestRunner
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

### Setup
The following environment variables should be created for the token and url first.
- REDCAP_API: The token assigned by your Redcap administrator
- REDCAP_URL: The URL associated with your institution's address to interact with the API
- Reference: https://www.google.com/search?q=create+environment+variables

Alternatively, you can manually overwrite the os.environ commands in each test with your own values, or access your sensitive data in some other more secure manner.
However, do not publish your token online!

By default, all Redcap protocols use record_id as the primary identifier for each participant.
If not using the default, define the alternative in your protocol below. Normally, this may be 'record_id'.

In [2]:
record_id = 'prescreening_id'

### Data Export Unit Tests

In [3]:
class TestDataExport(unittest.TestCase):
    """
        Test project data export
        Replace various fields, such as token, url, forms, and fields as needed to suit a given project
    """
    
    REDCAP_API = os.environ['REDCAP_API_BEECON_DEV']
    REDCAP_URL = os.environ['REDCAP_URL']  
    
    
    def test_basic_export(self):
        """
            Key Variables
            -------------
                threshold: Integer defining the minimum number of records expected
                fields_to_print_list: Modify to suit
            
        """
        threshold = 1
        fields_to_print_list = [record_id, 'randomization_id']
        result_limit = 3
        
        rcap = Redcapy(api_token=self.REDCAP_API, redcap_url=self.REDCAP_URL)
        return_value_json = rcap.export_records()

        if 'error' in return_value_json:  # no content
            print(return_value_json)
        else:
            for i, record in zip(range(result_limit), return_value_json):
                print({key: return_value_json[i].get(key) for key in fields_to_print_list})
        
        self.assertTrue(len(return_value_json) >= threshold, msg='Was expecting to see more than 1')
        
     
    def test_optional_fields_export(self):
        """
            Key Variables
            -------------
                threshold: Integer defining the minimum number of records expected
            
        """
        threshold = 1
        args = {}  # initializing dict of kwargs
        result_limit = 1

        forms_str = 'dental_screening'  # specify a form to limit output
        
        args['forms'] = forms_str
        args['rawOrLabel'] = 'label'
        args['rawOrLabelHeaders'] = 'label'
        
        rcap = Redcapy(api_token=self.REDCAP_API, redcap_url=self.REDCAP_URL)
        return_value_json = rcap.export_records(**args)

        if 'error' in return_value_json:  # no content
            print(return_value_json)
        else:
            for i, record in zip(range(result_limit), return_value_json):
                print({key: return_value_json[i].get(key) for key, value in record.items() })
        
        self.assertTrue(len(return_value_json) >= threshold, msg='Was expecting to see more than 1')
        

### Execute Export Tests

In [4]:
te = TestDataExport()
suite = TestLoader().loadTestsFromModule(te)
TextTestRunner().run(suite)

.

{'randomization_id': '1.001', 'prescreening_id': '101'}
{'randomization_id': '', 'prescreening_id': '101'}
{'randomization_id': '', 'prescreening_id': '101'}


.

{'ss02_untxcaries': "No  <b><font color='blue'>[No]</font></b>", 'ss04_dmftge7': "No  <b><font color='blue'>[No]</font></b>", 'mh01_dtxcomp': "No  <b><font color='blue'>[No]</font></b>", 'mh04_asthma': "No  <b><font color='blue'>[No]</font></b>", 'mh02_fvallergy': "No  <b><font color='blue'>[No]</font></b>", 'mh03_latexallergy': "No  <b><font color='blue'>[No]</font></b>", 'mh06_heartprob': "No  <b><font color='blue'>[No]</font></b>", 'mh09c_participate': '', 'ss01_numteeth': '0', 'mh04a_if_yes_explain': '', 'mh01b_type_dtxcomp': '', 'ss03_cariesexp': "No  <b><font color='blue'>[No]</font></b>", 'mh03a_gloves_available': '', 'mh07_chrondx': "No  <b><font color='blue'>[No]</font></b>", 'mh09_meds': "No  <b><font color='blue'>[No]</font></b>", 'mh04b_asthma': '', 'mh08_organxplt': "No  <b><font color='blue'>[No]</font></b>", 'ss07_notes': '', 'ss06_franklscore': '3 Positive', 'dental_screening_complete': 'Incomplete', 'mh05_tb': "No  <b><font color='blue'>[No]</font></b>", 'mh09b_typemed


----------------------------------------------------------------------
Ran 2 tests in 3.310s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

### MetaData Export Unit Tests

In [5]:
class TestMetaDataExport(unittest.TestCase):
    """
        
    """
    REDCAP_API = os.environ['REDCAP_API_BEECON_DEV']
    REDCAP_URL = os.environ['REDCAP_URL']   
    
    
    def test_basic_export(self):
        rcap = Redcapy(api_token=self.REDCAP_API, redcap_url=self.REDCAP_URL)

#         return_value_json = self.rcap.export_records(format='json', return_format='json', exportCheckboxLabel='true')
        return_value_json = rcap.export_data_dictionary()
        
        if 'error' in return_value_json:  # no content
            print(return_value_json)
        else:
            print('Partial output follows:\n')
            pprint.pprint(return_value_json[:2])
            
        threshold = 1
        self.assertTrue(len(return_value_json) > 1)  # Protocol presumed to have more than threshold events

### Execute MetaData Export Tests

In [6]:
t = TestMetaDataExport()
suite = TestLoader().loadTestsFromModule(t)
TextTestRunner().run(suite)

.

Partial output follows:

[{'branching_logic': '',
  'custom_alignment': '',
  'field_annotation': '',
  'field_label': 'Pre-Screening ID',
  'field_name': 'prescreening_id',
  'field_note': '',
  'field_type': 'text',
  'form_name': 'screening_eligibility',
  'identifier': '',
  'matrix_group_name': '',
  'matrix_ranking': '',
  'question_number': '',
  'required_field': '',
  'section_header': '',
  'select_choices_or_calculations': '',
  'text_validation_max': '',
  'text_validation_min': '',
  'text_validation_type_or_show_slider_number': ''},
 {'branching_logic': '',
  'custom_alignment': '',
  'field_annotation': '',
  'field_label': 'Date',
  'field_name': 'eligibility_date',
  'field_note': '',
  'field_type': 'text',
  'form_name': 'screening_eligibility',
  'identifier': '',
  'matrix_group_name': '',
  'matrix_ranking': '',
  'question_number': '',
  'required_field': 'y',
  'section_header': '',
  'select_choices_or_calculations': '',
  'text_validation_max': '',
  'text_val


----------------------------------------------------------------------
Ran 1 test in 1.010s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### Events Export Unit Tests

In [7]:
class TestEventExport(unittest.TestCase):
    """
        
    """
    REDCAP_API = os.environ['REDCAP_API_BEECON_DEV']
    REDCAP_URL = os.environ['REDCAP_URL']     
    
    
    def test_basic_export(self):
        rcap = Redcapy(api_token=self.REDCAP_API, redcap_url=self.REDCAP_URL)

        return_value_json = rcap.export_events()

        if 'error' in return_value_json:  # no content
            print(return_value_json)
        else:
            pprint.pprint(return_value_json)
            
        threshold = 1
        self.assertTrue(len(return_value_json) > threshold)  # Protocol presumed to have more than threshold events

### Execute Event Export Tests

In [8]:
t = TestEventExport()
suite = TestLoader().loadTestsFromModule(t)
TextTestRunner().run(suite)

.

[{'arm_num': '1',
  'custom_event_label': None,
  'day_offset': '0',
  'event_name': 'Screening',
  'offset_max': '0',
  'offset_min': '0',
  'unique_event_name': 'screening_arm_1'},
 {'arm_num': '2',
  'custom_event_label': '00_week_baseline',
  'day_offset': '0',
  'event_name': 'Baseline',
  'offset_max': '0',
  'offset_min': '0',
  'unique_event_name': 'baseline_arm_2'},
 {'arm_num': '2',
  'custom_event_label': '10_week_followup1',
  'day_offset': '70',
  'event_name': 'Follow-up 1',
  'offset_max': '30',
  'offset_min': '30',
  'unique_event_name': 'followup_1_arm_2'},
 {'arm_num': '2',
  'custom_event_label': '99_week_floating',
  'day_offset': '71',
  'event_name': 'Floating',
  'offset_max': '0',
  'offset_min': '0',
  'unique_event_name': 'floating_arm_2'},
 {'arm_num': '3',
  'custom_event_label': '00_week_b',
  'day_offset': '0',
  'event_name': 'Baseline',
  'offset_max': '0',
  'offset_min': '0',
  'unique_event_name': 'baseline_arm_3'},
 {'arm_num': '3',
  'custom_event_


----------------------------------------------------------------------
Ran 1 test in 0.566s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### Data Import Unit Test Class

In [9]:
class TestImport(unittest.TestCase):
    """
        return_key can be 'ids' or 'count'

        Fields defined in each import obviously should exist in the project before attempting.
        
    """
    REDCAP_API = os.environ['REDCAP_API_BEECON_DEV']
    REDCAP_URL = os.environ['REDCAP_URL']      


    def __import_records__(self, data_to_upload, returnContent):
        """
            Common private code for the import unit tests
            
            Parameters
            ----------
                data_to_upload_list: a list of records to import, where each record is a dict
        
        """
        rcap = Redcapy(api_token=self.REDCAP_API, redcap_url=self.REDCAP_URL)
        
        if type(data_to_upload) == dict:  # a dict is presumed to be a single record
            data_to_upload_list = [data_to_upload]
        else:
            data_to_upload_list = data_to_upload
            
        data_to_upload_list_count = len(data_to_upload_list)
        data_to_upload_str = json.dumps(data_to_upload_list)
        
        return_value_json = rcap.import_records(data_to_upload=data_to_upload_str, 
                                                returnFormat='json',
                                                returnContent=returnContent)

        return return_value_json
          
        
    def test_malformed_record_id_import(self):
        """
            Attempt to import a malformed id
            
            data_to_upload_list fields should be modified to reflect your protocol's data
            data_to_upload_list is a list of dicts, to be converted into a json object
            
            Key Variables
            -------------
                data_to_upload_list: list of dict study key/value pairs                
            
            On error, expect to see the following json (generated by the server): 
                {"error": "The participant id field (record_id) is missing."}
        """
        print('Begin test_malformed_record_id_import test') 
        
        data_to_upload_list = [{'record_i': '2', 'subject_id': '435', 'redcap_event_name': 'screening_arm_1'},
                          {'record_i': '1', 'subject_id': '45', 'redcap_event_name': 'screening_arm_1'},
                          {record_id: '3', 'subject_id': '3', 'redcap_event_name': 'screening_arm_1'}]    
        
        rcap = Redcapy(api_token=self.REDCAP_API, redcap_url=self.REDCAP_URL)        
        data_to_upload_str = json.dumps(data_to_upload_list)
        return_value_json = rcap.import_records(data_to_upload=data_to_upload_str)

        self.assertTrue('error' in return_value_json, 
                        msg='test_malformed_record_id_import method did not throw an error upon import')
        print('End test_malformed_record_id_import test\n')
      
    def test_iterative_import(self):   
        """
            Import a list of records individually, rather than as a batch upload
            
            Key Variables
            -------------
                return_key: id or count
                data_to_upload_list: list of dict study key/value pairs
                
        """
        print('Begin test_iterative_import test')
        
        return_key = 'count'
        create_record_count = 16
        base_consent_date = date(2017, 6, 1)
        
        # Generate a list of records to upload
        data_to_upload_list = [{record_id: str(i+101), 
                                'redcap_event_name': 'screening_arm_1', 
                                'ss01_numteeth': i, 
                                'consent_date': (base_consent_date + timedelta(days=i)).strftime('%Y-%m-%d') }
                               for i in range(create_record_count)
                              ]
        print('List of records to import into Redcap:')
        pprint.pprint(data_to_upload_list)
                
        data_to_upload_list_count = len(data_to_upload_list)
        return_count = 0
        import_attempt_count = 0
        
        for i, record in enumerate(data_to_upload_list):
            import_list = [data_to_upload_list[i]]
            import_attempt_count += 1
            return_value_json = self.__import_records__(data_to_upload=import_list, returnContent=return_key) 

            if 'count' in return_value_json and return_value_json['count'] == 1:
                return_count += 1
            elif 'error' in return_value_json:
                if record_id in import_list[0]:
                    print(record_id, import_list, ' failed to import.')
                else:
                    print('Test record does not have a', 'record id')
                    
                print('Error returned from server:', return_value_json['error'])
            else:
                print(record_id, import_list, ' failed to import.')
                print('Server returned : ', return_value_json)                
                
        print('{} distinct imports attempted, {} successes from a list of {} records'.format(import_attempt_count, 
                                                                                    return_count, 
                                                                                    data_to_upload_list_count))
        self.assertEqual(data_to_upload_list_count, return_count, msg='At least one record was not imported properly.')
        print('End test_iterative_import test\n')
        
    def test_basic_import_delete(self):
        """
            First create a record using a randomly generated ID, then delete it
            
            Key Variables
            -------------
                id = the record_id to create, then delete
                return_key: id or count
                data_to_upload_list: list of dict study key/value pairs            
        """
        print('Begin test_basic_import_delete test')
        id = str(hashlib.sha1().hexdigest()[:16])
        return_key = 'count'
        
        rcap = Redcapy(api_token=self.REDCAP_API, redcap_url=self.REDCAP_URL)
        
        # Note: code requires modifications if the list is extended to length greater than 1
        data_to_upload_list = [{record_id: id, 'redcap_event_name': 'screening_arm_1'}]
        
        data_to_upload_list_count = len(data_to_upload_list)
        return_count = 0
        import_attempt_count = 0

        for i, record in enumerate(data_to_upload_list):
            data_to_upload = [record]
            import_attempt_count += 1
            
            print('Importing record ID', id)
            
            return_value_json = self.__import_records__(data_to_upload=data_to_upload, returnContent=return_key) 

            if 'count' in return_value_json and return_value_json['count'] == 1:
                return_count += 1
                print('ID', id, 'successfully imported.')
            elif 'error' in return_value_json:
                if record_id in data_to_upload:
                    print(record_id, data_to_upload[record_id], ' failed to import.')
                else:
                    print('Test record does not have a record id')
                    
                print('Error returned from server:', return_value_json['error'])
            else:
                print(record_id, data_to_upload[record_id], ' failed to import.')
                print('Server returned : ', return_value_json)    
                
            print('{} distinct imports attempted, {} successes from a list of {} records'.format(import_attempt_count, 
                                                                                    return_count, 
                                                                                    data_to_upload_list_count))                
            if return_count == 1:
                wait_seconds = 10
                
                print('Attempting to delete record for ID', id)
                print('Please wait {} seconds to complete database delete operation'.format(wait_seconds))
                print('Wait time may require adjustment if server is busy')
                
                delete_return_count = rcap.delete_record(id_to_delete=id)
                
                time.sleep(wait_seconds)
            
            if delete_return_count == 1:
                print('ID', id, 'successfully deleted.')
            elif return_count == 1:
                print('ID', id, 'was imported but not successfully deleted.')
                
        self.assertTrue(data_to_upload_list_count == return_count and delete_return_count == return_count, 
                        msg='At least one record was not imported properly.')

        print('End test_basic_import_delete test\n') 

### Execute Import Tests

In [10]:
# Note the order of operations may differ from how they appear in the test
t = TestImport()
suite = TestLoader().loadTestsFromModule(t)
TextTestRunner().run(suite)

Begin test_basic_import_delete test
Importing record ID da39a3ee5e6b4b0d
ID da39a3ee5e6b4b0d successfully imported.
1 distinct imports attempted, 1 successes from a list of 1 records
Attempting to delete record for ID da39a3ee5e6b4b0d
Please wait 10 seconds to complete database delete operation
Wait time may require adjustment if server is busy


.

ID da39a3ee5e6b4b0d successfully deleted.
End test_basic_import_delete test

Begin test_iterative_import test
List of records to import into Redcap:
[{'consent_date': '2017-06-01',
  'prescreening_id': '101',
  'redcap_event_name': 'screening_arm_1',
  'ss01_numteeth': 0},
 {'consent_date': '2017-06-02',
  'prescreening_id': '102',
  'redcap_event_name': 'screening_arm_1',
  'ss01_numteeth': 1},
 {'consent_date': '2017-06-03',
  'prescreening_id': '103',
  'redcap_event_name': 'screening_arm_1',
  'ss01_numteeth': 2},
 {'consent_date': '2017-06-04',
  'prescreening_id': '104',
  'redcap_event_name': 'screening_arm_1',
  'ss01_numteeth': 3},
 {'consent_date': '2017-06-05',
  'prescreening_id': '105',
  'redcap_event_name': 'screening_arm_1',
  'ss01_numteeth': 4},
 {'consent_date': '2017-06-06',
  'prescreening_id': '106',
  'redcap_event_name': 'screening_arm_1',
  'ss01_numteeth': 5},
 {'consent_date': '2017-06-07',
  'prescreening_id': '107',
  'redcap_event_name': 'screening_arm_1',

.

16 distinct imports attempted, 16 successes from a list of 16 records
End test_iterative_import test

Begin test_malformed_record_id_import test


.

End test_malformed_record_id_import test




----------------------------------------------------------------------
Ran 3 tests in 20.928s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>